In [1]:
from seeq import spy
import pandas as pd
import numpy as np

In [1]:
# Log into Seeq Server if you're not using Seeq Data Lab:
spy.login(url='http://localhost:34216', credentials_file='../../credentials.key', force=False)

In [3]:
from seeq.spy.assets import Asset

class Area(Asset):
    def initialize(self):
        # These are effectively x,y coordinates proportional to the
        # map of the U.S. in this folder.
        map_positions = {
            # Cooling Tower 1
            'A': ['15%', '4%'],
            'B': ['45%', '60%'],
            'C': ['35%', '35%'],
            'G': ['70%', '60%'],
            'H': ['32%', '80%'],
            'I': ['55%', '40%'],
            'J': ['15%', '34%'],
            'K': ['80%', '6%'],
            
            # Cooling Tower 2
            'D': ['40%', '50%'],
            'E': ['50%', '60%'],
            'F': ['60%', '70%']
        }
        
        self.map_position = map_positions[self['Name']]
    
    @Asset.Attribute()
    def Incidents(self, metadata):
        return {
            'Type': 'Condition',
            'Formula': '$a.validValues().valueSearch(isGreaterThan(120))',
            'Formula Parameters': {
                '$a': metadata[metadata['Name'] == 'Temperature']
            }
        }
    
    @Asset.Attribute()
    def Incident_Count(self, metadata):
        return {
            'Type': 'Metric',
            'Measured Item': self.Incidents(),
            'Measured Item Maximum Duration': '40d',
            'Statistic': 'count',
            'Thresholds': {
                'HiHiHi': 1,
                'HiHi': 0
            }
        }
    
    @Asset.Display()
    def Incident_Display(self, metadata, analysis):
        worksheet = analysis.worksheet('Area ' + self.definition['Name'])
        
        workstep = worksheet.workstep()
        workstep.display_items = [{
            'Item': self.Incident_Count()
        }]
        workstep.view = 'Scorecard'
        workstep.data['state']['stores']['sqTrendMetricStore']['scorecardHeaders'] = {
          "type": "none"
        }
        workstep.data['state']['stores']['sqTrendMetricStore']['scorecardColumns'] = list()
        
        return workstep
    
    @Asset.Display()
    def Detail(self, metadata, analysis):
        worksheet = analysis.worksheet('Area ' + self.definition['Name'])
        
        workstep = worksheet.workstep()
        workstep.display_items = metadata[metadata['Name'] == 'Temperature']
        workstep.view = 'Trend'
        
        return workstep

class Cooling_Tower(Asset):
    @Asset.Component()
    def Areas(self, metadata):
        return self.build_components(Area, metadata, column_name='Area')
    
    @Asset.DateRange()
    def Last_Two_Months(self, metadata):
        return {
            'Start': '2019-11-01',
            'End': '2020-01-01'
        }
    
    @Asset.DateRange()
    def Last_Week(self, metadata):
        return {
            'Start': '2020-01-01',
            'End': '2020-01-08'
        }
        
    @Asset.Document()
    def Topic_Document(self, metadata, topic):
        document = topic.document('Cooling Tower ' + self.definition['Name'])
        document.render_template(filename='Cooling_Tower_Health.html', asset=self)
        
class Region(Asset):    
    @Asset.Component()
    def Cooling_Towers(self, metadata):
        return self.build_components(Cooling_Tower, metadata, column_name='Cooling Tower')

In [4]:
search_df = spy.search({
    'Path': 'Example',
    'Type': 'Signal',
    'Name': 'Temperature'})

search_df

,Count,Time
Results,10,00:00:00.23


,ID,Path,Asset,Name,Description,Type,Value Unit Of Measure,Datasource Name
0,1CBC2C96-A305-4D9E-BAA8-0CBF269C3726,Example >> Cooling Tower 1,Area C,Temperature,NaN,StoredSignal,°F,Example Data
1,AA6599FA-BA60-433E-A2CA-459341F66D5C,Example >> Cooling Tower 1,Area I,Temperature,NaN,StoredSignal,°F,Example Data
2,0F6F884A-C4AE-47D2-AF90-D09F9B40E5F1,Example >> Cooling Tower 1,Area G,Temperature,NaN,StoredSignal,°F,Example Data
3,027A1C73-4564-40C2-98CE-16A0A93EFE66,Example >> Cooling Tower 2,Area E,Temperature,NaN,StoredSignal,°F,Example Data
4,D397B18E-E93C-4481-B035-1704BA7DF893,Example >> Cooling Tower 1,Area B,Temperature,NaN,StoredSignal,°F,Example Data
5,F9F8F062-97D4-4D99-9B18-5EFB6F1C2243,Example >> Cooling Tower 1,Area J,Temperature,NaN,StoredSignal,°F,Example Data
6,0A0D4C30-49B1-49CD-BFF3-00669E53FBFD,Example >> Cooling Tower 1,Area K,Temperature,NaN,StoredSignal,°F,Example Data
7,C63DCBDC-6DFE-42AE-BD56-3EA290E20C5B,Example >> Cooling Tower 1,Area H,Temperature,NaN,StoredSignal,°F,Example Data
8,5B72793B-9680-4B98-BBF0-CEF12343261E,Example >> Cooling Tower 2,Area D,Temperature,NaN,StoredSignal,°F,Example Data
9,683778EC-678A-4FC4-BBCA-B835D180B042,Example >> Cooling Tower 1,Area A,Temperature,NaN,StoredSignal,°F,Example Data


In [5]:
metadata_df = search_df.copy()
metadata_df['Cooling Tower'] = metadata_df['Path'].str.extract(r'Cooling Tower (\d)')
metadata_df['Area'] = metadata_df['Asset'].str.extract(r'Area (\w)')
metadata_df['Build Path'] = 'Cooling Tower Health'
metadata_df['Build Asset'] = 'North America'
metadata_df

,ID,Path,Asset,Name,Description,Type,Value Unit Of Measure,Datasource Name,Cooling Tower,Area,Build Path,Build Asset
0,1CBC2C96-A305-4D9E-BAA8-0CBF269C3726,Example >> Cooling Tower 1,Area C,Temperature,NaN,StoredSignal,°F,Example Data,1,C,Cooling Tower Health,North America
1,AA6599FA-BA60-433E-A2CA-459341F66D5C,Example >> Cooling Tower 1,Area I,Temperature,NaN,StoredSignal,°F,Example Data,1,I,Cooling Tower Health,North America
2,0F6F884A-C4AE-47D2-AF90-D09F9B40E5F1,Example >> Cooling Tower 1,Area G,Temperature,NaN,StoredSignal,°F,Example Data,1,G,Cooling Tower Health,North America
3,027A1C73-4564-40C2-98CE-16A0A93EFE66,Example >> Cooling Tower 2,Area E,Temperature,NaN,StoredSignal,°F,Example Data,2,E,Cooling Tower Health,North America
4,D397B18E-E93C-4481-B035-1704BA7DF893,Example >> Cooling Tower 1,Area B,Temperature,NaN,StoredSignal,°F,Example Data,1,B,Cooling Tower Health,North America
5,F9F8F062-97D4-4D99-9B18-5EFB6F1C2243,Example >> Cooling Tower 1,Area J,Temperature,NaN,StoredSignal,°F,Example Data,1,J,Cooling Tower Health,North America
6,0A0D4C30-49B1-49CD-BFF3-00669E53FBFD,Example >> Cooling Tower 1,Area K,Temperature,NaN,StoredSignal,°F,Example Data,1,K,Cooling Tower Health,North America
7,C63DCBDC-6DFE-42AE-BD56-3EA290E20C5B,Example >> Cooling Tower 1,Area H,Temperature,NaN,StoredSignal,°F,Example Data,1,H,Cooling Tower Health,North America
8,5B72793B-9680-4B98-BBF0-CEF12343261E,Example >> Cooling Tower 2,Area D,Temperature,NaN,StoredSignal,°F,Example Data,2,D,Cooling Tower Health,North America
9,683778EC-678A-4FC4-BBCA-B835D180B042,Example >> Cooling Tower 1,Area A,Temperature,NaN,StoredSignal,°F,Example Data,1,A,Cooling Tower Health,North America


In [6]:
build_df = spy.assets.build(Region, metadata_df)
build_df

J:\BitBucket\crab\sdk\pypi\seeq\spy\workbooks\_worksheet.py:579: FutureWarning: Passing a datetime or Timestamp with tzinfo and the tz parameter will raise in the future. Use tz_convert instead.
  end_datetime = pd.Timestamp(existing_date_range["End"], tz=_login.get_user_timezone())


,Asset,Asset Object,Build Result,Formula,Formula Parameters,Measured Item,Measured Item Maximum Duration,Name,Object,Path,Statistic,Template,Thresholds,Type,Workbook Type
0,C,Cooling Tower Health >> North America >> 1 >> C,Success,$a.validValues().valueSearch(isGreaterThan(120)),{'$a': ['ID']},NaN,NaN,Incidents,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,Condition,NaN
1,C,Cooling Tower Health >> North America >> 1 >> C,Success,NaN,NaN,"{'Type': 'Condition', 'Formula': '$a.validValu...",40d,Incident Count,NaN,Cooling Tower Health >> North America >> 1,count,Area,"{'HiHiHi': 1, 'HiHi': 0}",Metric,NaN
2,C,Cooling Tower Health >> North America >> 1 >> C,Success,NaN,NaN,NaN,NaN,C,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,Asset,NaN
3,I,Cooling Tower Health >> North America >> 1 >> I,Success,$a.validValues().valueSearch(isGreaterThan(120)),{'$a': ['ID']},NaN,NaN,Incidents,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,Condition,NaN
4,I,Cooling Tower Health >> North America >> 1 >> I,Success,NaN,NaN,"{'Type': 'Condition', 'Formula': '$a.validValu...",40d,Incident Count,NaN,Cooling Tower Health >> North America >> 1,count,Area,"{'HiHiHi': 1, 'HiHi': 0}",Metric,NaN
5,I,Cooling Tower Health >> North America >> 1 >> I,Success,NaN,NaN,NaN,NaN,I,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,Asset,NaN
6,G,Cooling Tower Health >> North America >> 1 >> G,Success,$a.validValues().valueSearch(isGreaterThan(120)),{'$a': ['ID']},NaN,NaN,Incidents,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,Condition,NaN
7,G,Cooling Tower Health >> North America >> 1 >> G,Success,NaN,NaN,"{'Type': 'Condition', 'Formula': '$a.validValu...",40d,Incident Count,NaN,Cooling Tower Health >> North America >> 1,count,Area,"{'HiHiHi': 1, 'HiHi': 0}",Metric,NaN
8,G,Cooling Tower Health >> North America >> 1 >> G,Success,NaN,NaN,NaN,NaN,G,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,Asset,NaN
9,B,Cooling Tower Health >> North America >> 1 >> B,Success,$a.validValues().valueSearch(isGreaterThan(120)),{'$a': ['ID']},NaN,NaN,Incidents,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,Condition,NaN


In [7]:
spy.push(metadata=build_df)

,Asset,Condition,Overall,Relationship,Scalar,Signal,Threshold Metric
Items pushed,14,10,43,33,0,0,10


,Asset,Asset Object,Build Result,Formula,Formula Parameters,Measured Item,Measured Item Maximum Duration,Name,Object,Path,Statistic,Template,Thresholds,Type,Workbook Type,Datasource Class,Datasource ID,Data ID,ID,Push Result
0,C,Cooling Tower Health >> North America >> 1 >> C,Success,$a.validValues().valueSearch(isGreaterThan(120)),[a=1CBC2C96-A305-4D9E-BAA8-0CBF269C3726],NaN,NaN,Incidents,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,CalculatedCondition,NaN,Seeq Data Lab,Seeq Data Lab,[B15567E0-C146-48F4-B7E2-A1ABBB58FBB0] {Condit...,DB04CB0E-2E3E-4E9C-B2B1-32B8E591906D,Success
1,C,Cooling Tower Health >> North America >> 1 >> C,Success,NaN,NaN,"{'Type': 'Condition', 'Formula': '$a.validValu...",40d,Incident Count,NaN,Cooling Tower Health >> North America >> 1,count,Area,"{'HiHiHi': 1, 'HiHi': 0}",ThresholdMetric,NaN,Seeq Data Lab,Seeq Data Lab,[B15567E0-C146-48F4-B7E2-A1ABBB58FBB0] {Metric...,889FCAEC-26B1-4919-80A4-6B32D0514931,Success
2,C,Cooling Tower Health >> North America >> 1 >> C,Success,NaN,NaN,NaN,NaN,C,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,Asset,NaN,Seeq Data Lab,Seeq Data Lab,[B15567E0-C146-48F4-B7E2-A1ABBB58FBB0] {Asset}...,206EA1CB-3644-47AB-B7EE-240F225B3107,Success
3,I,Cooling Tower Health >> North America >> 1 >> I,Success,$a.validValues().valueSearch(isGreaterThan(120)),[a=AA6599FA-BA60-433E-A2CA-459341F66D5C],NaN,NaN,Incidents,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,CalculatedCondition,NaN,Seeq Data Lab,Seeq Data Lab,[B15567E0-C146-48F4-B7E2-A1ABBB58FBB0] {Condit...,4B2F1252-0485-43D3-912F-B81928F161DE,Success
4,I,Cooling Tower Health >> North America >> 1 >> I,Success,NaN,NaN,"{'Type': 'Condition', 'Formula': '$a.validValu...",40d,Incident Count,NaN,Cooling Tower Health >> North America >> 1,count,Area,"{'HiHiHi': 1, 'HiHi': 0}",ThresholdMetric,NaN,Seeq Data Lab,Seeq Data Lab,[B15567E0-C146-48F4-B7E2-A1ABBB58FBB0] {Metric...,B5E155F5-266B-47BE-9C27-6D8AE7422A63,Success
5,I,Cooling Tower Health >> North America >> 1 >> I,Success,NaN,NaN,NaN,NaN,I,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,Asset,NaN,Seeq Data Lab,Seeq Data Lab,[B15567E0-C146-48F4-B7E2-A1ABBB58FBB0] {Asset}...,0696873B-4936-4F99-9077-D17F9BF6308F,Success
6,G,Cooling Tower Health >> North America >> 1 >> G,Success,$a.validValues().valueSearch(isGreaterThan(120)),[a=0F6F884A-C4AE-47D2-AF90-D09F9B40E5F1],NaN,NaN,Incidents,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,CalculatedCondition,NaN,Seeq Data Lab,Seeq Data Lab,[B15567E0-C146-48F4-B7E2-A1ABBB58FBB0] {Condit...,9E72AC6B-C2AC-4EE2-A1F5-47E3F33B99A2,Success
7,G,Cooling Tower Health >> North America >> 1 >> G,Success,NaN,NaN,"{'Type': 'Condition', 'Formula': '$a.validValu...",40d,Incident Count,NaN,Cooling Tower Health >> North America >> 1,count,Area,"{'HiHiHi': 1, 'HiHi': 0}",ThresholdMetric,NaN,Seeq Data Lab,Seeq Data Lab,[B15567E0-C146-48F4-B7E2-A1ABBB58FBB0] {Metric...,310C0814-B8A4-479D-BACA-CBBF8CA6F528,Success
8,G,Cooling Tower Health >> North America >> 1 >> G,Success,NaN,NaN,NaN,NaN,G,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,Asset,NaN,Seeq Data Lab,Seeq Data Lab,[B15567E0-C146-48F4-B7E2-A1ABBB58FBB0] {Asset}...,5058FF61-FAB5-4478-AC2C-34C755F1C1E1,Success
9,B,Cooling Tower Health >> North America >> 1 >> B,Success,$a.validValues().valueSearch(isGreaterThan(120)),[a=D397B18E-E93C-4481-B035-1704BA7DF893],NaN,NaN,Incidents,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,CalculatedCondition,NaN,Seeq Data Lab,Seeq Data Lab,[B15567E0-C146-48F4-B7E2-A1ABBB58FBB0] {Condit...,947C8F9F-03BD-44FA-92F3-F10F09EEAA62,Success
